In [1]:
import requests, base64, time, json
OM2M_URL = "http://127.0.0.1:8080/~"
CSE_ID = "/in-cse/"
CSE_NAME = "in-name"
LOGIN="admin"
PSWD="admin"
OM2M_BASE = OM2M_URL+CSE_ID
auth_headers = {"X-M2M-ORIGIN":LOGIN+":"+PSWD}
# The other accepted value is application/xml
common_headers = {"Accept": "application/json"}

def create_AE(name):
    header_ae = {"Content-Type":"application/xml;ty=2"}
    name_ae = name
    body_ae = """
    <m2m:ae xmlns:m2m="http://www.onem2m.org/xml/protocols" rn="{0}">
        <api>semantic</api>
        <lbl>Type/semantic Category/temperature Location/home</lbl>
        <rr>true</rr>
    </m2m:ae>
    """.format(name_ae)
    response = requests.delete(OM2M_BASE+CSE_NAME+"/"+name_ae, headers={**auth_headers, **common_headers})
    print(response)
    response = requests.post(OM2M_BASE, data=body_ae, headers={**auth_headers, **common_headers, **header_ae})
    print(response)
def create_CNT(name_ae, name_cnt):
    header_cnt = {"Content-Type":"application/xml;ty=3"}
    body_cnt = """<m2m:cnt xmlns:m2m="http://www.onem2m.org/xml/protocols" rn="{0}"></m2m:cnt>""".format(name_cnt)
    print(body_cnt)
    response = requests.post(OM2M_BASE+CSE_NAME+"/"+name_ae, data=body_cnt, headers={**auth_headers, **common_headers, **header_cnt})
    print(response)


In [2]:
create_AE("Fish_Tank")
time.sleep(2)
create_CNT("Fish_Tank", "Turbidity_Sensor")


<Response [404]>
<Response [201]>
<m2m:cnt xmlns:m2m="http://www.onem2m.org/xml/protocols" rn="Turbidity_Sensor"></m2m:cnt>
<Response [201]>


In [3]:
import xml.sax.saxutils as saxutils
def create_CIN_message(ae_name, cnt_name, inner_obj_str):
    #XML-escape inner_obj_str
    escaped = saxutils.escape(inner_obj_str, {'"': '&quot;', "'": '&apos;'})
    #例如 '&lt;obj&gt;&lt;str name=&quot;Turbidity&quot; val=&quot;5&quot;/&gt;&lt;/obj&gt;'

    root_ns = 'xmlns:m2m="http://www.onem2m.org/xml/protocols"'

    # 4. 組 XML body
    body = f"""<m2m:cin {root_ns}>
    <cnf>message</cnf>
    <con>{escaped}</con>
    </m2m:cin>"""

    #print(body)

    # POST
    url = OM2M_BASE + CSE_NAME + f"/{ae_name}/{cnt_name}"
    headers = {
        **auth_headers,
        "Content-Type": "application/xml;ty=4",
        **common_headers
    }
    resp = requests.post(url, data=body.encode('utf-8'), headers=headers)
    # debug
    #print("HTTP status:", resp.status_code)
    #print("Response text:", resp.text)


In [4]:
import xml.etree.ElementTree as ET
def get_actuator():
    resp = requests.get(OM2M_BASE + CSE_NAME + "/Fish_Tank/Actuator/la",headers={**auth_headers, **common_headers})
    
    data = resp.json()
    cin = data.get("m2m:cin") or data.get("cin")
    con = cin.get("con")
    inner_xml = saxutils.unescape(con, {'&quot;': '"', '&apos;': "'"})
    root = ET.fromstring(inner_xml)
    elem = root.find('.//bool')
    val = elem.attrib.get('val')
    return val

In [5]:
i = 10
while True:
    if(get_actuator() == '1'):
        print('Start changing the fish tank water')
        inner = f'<obj><bool name="ON" val="0"/></obj>'
        create_CIN_message(ae_name="Fish_Tank",cnt_name="Actuator",inner_obj_str=inner)
        time.sleep(3)
        i = 10
        print('Fish tank water change completed')
        
    else:
        inner = f'<obj><str name="Turbidity" val="{i}"/></obj>'
        create_CIN_message(ae_name="Fish_Tank",cnt_name="Turbidity_Sensor",inner_obj_str=inner)
        i += 10  
        time.sleep(3)  


Start changing the fish tank water
Fish tank water change completed


KeyboardInterrupt: 